In [1]:
!pip install transformers==4.36.2
!pip install peft==0.7.1
!pip install accelerate==0.25.0
!pip install datasets==2.15.0
!pip install tqdm==4.66.1
!pip install wandb==0.16.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which 

In [2]:
!accelerate estimate-memory t5-small --library_name transformers

Loading pretrained config for `t5-small` from `transformers`...
config.json: 100% 1.21k/1.21k [00:00<00:00, 4.98MB/s]
┌────────────────────────────────────────────────────┐
│        Memory Usage for loading `t5-small`         │
├───────┬─────────────┬──────────┬───────────────────┤
│ dtype │Largest Layer│Total Size│Training using Adam│
├───────┼─────────────┼──────────┼───────────────────┤
│float32│   62.75 MB  │230.81 MB │     923.26 MB     │
│float16│   31.38 MB  │115.41 MB │     461.63 MB     │
│  int8 │   15.69 MB  │ 57.7 MB  │     230.81 MB     │
│  int4 │   7.84 MB   │ 28.85 MB │     115.41 MB     │
└───────┴─────────────┴──────────┴───────────────────┘


In [3]:
from datasets import load_dataset

dataset_name="financial_phrasebank"
subset_name="sentences_allagree"

dataset=load_dataset(dataset_name, subset_name)
dataset=dataset["train"].train_test_split(test_size=0.1)
dataset["validation"]=dataset["test"]
del dataset["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [4]:
classes=dataset["train"].features["label"].names
dataset=dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

In [5]:
dataset["train"][1]

{'sentence': 'Earnings per share were higher at 0.48 against 0.37 a year before and ahead of market consensus of 0.40 eur .',
 'label': 2,
 'text_label': 'positive'}

In [6]:
from transformers import AutoTokenizer

model_name="t5-small"

tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
text_column="sentence"
label_column="text_label"
max_length=128
batch_size=8

def preprocess_function(examples):
    inputs=examples[text_column]
    targets=examples[label_column]
    model_inputs=tokenizer(
        inputs,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    labels=tokenizer(
        targets,
        max_length=2,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    labels=labels["input_ids"]
    labels[labels==tokenizer.pad_token_id]=-100 # ignore tokenizer pad token in the loss
    model_inputs["labels"]=labels
    return model_inputs

processed_datasets=dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [8]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset=processed_datasets["train"]
eval_dataset=processed_datasets["validation"]

train_dataloader=DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)

eval_dataloader=DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)

In [9]:
from peft import TaskType, get_peft_model
from peft import PrefixTuningConfig
from transformers import AutoModelForSeq2SeqLM

peft_config=PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=20
)

model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
model=get_peft_model(model, peft_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 122,880 || all params: 60,629,504 || trainable%: 0.20267360260773368


In [10]:
import torch
from transformers import get_linear_schedule_with_warmup

num_epochs=5

optimizer=torch.optim.AdamW(model.parameters(), lr=1e-2)
lr_scheduler=get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader)*num_epochs),
)

In [11]:
from accelerate import Accelerator
from tqdm import tqdm

accelerator=Accelerator(log_with="wandb")
accelerator.init_trackers("prefix_tuning")

model, optimizer, train_dataloader, eval_dataloader, lr_scheduler=accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

device=accelerator.device
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss=0

    for step, batch in enumerate(tqdm(train_dataloader)):
        batch={k:v.to(device) for k,v in batch.items()}
        outputs=model(**batch)
        loss=outputs.loss

        accelerator.log({"step":step, "training_loss":loss})
        total_loss+=loss.detach().float()

        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss=0
    eval_preds=[]
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch={k:v.to(device) for k,v in batch.items()}
        with torch.no_grad():
            outputs=model(**batch)
        loss=outputs.loss

        accelerator.log({"eval_loss":loss})
        eval_loss+=loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(
                torch.argmax(outputs.logits,-1).detach().cpu().numpy(),
                skip_special_tokens=True
            )
        )
    eval_epoch_loss=eval_loss/len(eval_dataloader)
    eval_ppl=torch.exp(eval_epoch_loss)
    train_epoch_loss=total_loss/len(train_dataloader)
    train_ppl=torch.exp(train_epoch_loss)
    accelerator.print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")
accelerator.end_training()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


100%|██████████| 29/29 [00:00<00:00, 48.14it/s]


...

100%|██████████| 29/29 [00:00<00:00, 50.59it/s]


epoch=3: train_ppl=tensor(1.2973, device='cuda:0') train_epoch_loss=tensor(0.2603, device='cuda:0') eval_ppl=tensor(1.1599, device='cuda:0') eval_epoch_loss=tensor(0.1483, device='cuda:0')


100%|██████████| 29/29 [00:00<00:00, 47.50it/s]


epoch=4: train_ppl=tensor(1.2886, device='cuda:0') train_epoch_loss=tensor(0.2536, device='cuda:0') eval_ppl=tensor(1.1641, device='cuda:0') eval_epoch_loss=tensor(0.1520, device='cuda:0')


eval_loss,█▆▆▃▄█▇▃▆▄▅▅▃▃▂▃▆▂▄▃▂▃▇▃▅▃▆▄▃▂▁▃▆▂▄▃▂▃▅▁
step,▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█
training_loss,█▃▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_loss,0.02177
step,254
training_loss,0.75327


In [12]:
correct=0
total=0
for pred,true in zip(eval_preds,dataset["validation"]["text_label"]):
    if pred.strip()==true.strip():
        correct+=1
    total+=1
accuracy=correct/total*100
accuracy

89.42731277533039